<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

In [ ]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import min_weight_full_bipartite_matching

Let us first consider an example in which all weights are equal:


In [ ]:
biadjacency_matrix = csr_matrix([[1, 1, 1], [1, 0, 0], [0, 1, 0]])

Here, all we get is a perfect matching of the graph:


In [ ]:
print(min_weight_full_bipartite_matching(biadjacency_matrix)[1])

[2 0 1]

That is, the first, second, and third rows are matched with the third,
first, and second column respectively. Note that in this example, the 0
in the input matrix does *not* correspond to an edge with weight 0, but
rather a pair of vertices not paired by an edge.

Note also that in this case, the output matches the result of applying
:func:`maximum_bipartite_matching`:


In [ ]:
from scipy.sparse.csgraph import maximum_bipartite_matching
biadjacency = csr_matrix([[1, 1, 1], [1, 0, 0], [0, 1, 0]])
print(maximum_bipartite_matching(biadjacency, perm_type='column'))

[2 0 1]

When multiple edges are available, the ones with lowest weights are
preferred:


In [ ]:
biadjacency = csr_matrix([[3, 3, 6], [4, 3, 5], [10, 1, 8]])
row_ind, col_ind = min_weight_full_bipartite_matching(biadjacency)
print(col_ind)

[0 2 1]

The total weight in this case is $3 + 5 + 1 = 9$:


In [ ]:
print(biadjacency[row_ind, col_ind].sum())

9

When the matrix is not square, i.e. when the two partitions have different
cardinalities, the matching is as large as the smaller of the two
partitions:


In [ ]:
biadjacency = csr_matrix([[0, 1, 1], [0, 2, 3]])
row_ind, col_ind = min_weight_full_bipartite_matching(biadjacency)
print(row_ind, col_ind)

[0 1] [2 1]

In [ ]:
biadjacency = csr_matrix([[0, 1], [3, 1], [1, 4]])
row_ind, col_ind = min_weight_full_bipartite_matching(biadjacency)
print(row_ind, col_ind)

[0 2] [1 0]

When one or both of the partitions are empty, the matching is empty as
well:


In [ ]:
biadjacency = csr_matrix((2, 0))
row_ind, col_ind = min_weight_full_bipartite_matching(biadjacency)
print(row_ind, col_ind)

[] []

In general, we will always reach the same sum of weights as if we had used
:func:`scipy.optimize.linear_sum_assignment` but note that for that one,
missing edges are represented by a matrix entry of ``float('inf')``. Let us
generate a random sparse matrix with integer entries between 1 and 10:


In [ ]:
import numpy as np
from scipy.sparse import random
from scipy.optimize import linear_sum_assignment
sparse = random(10, 10, random_state=42, density=.5, format='coo') * 10
sparse.data = np.ceil(sparse.data)
dense = sparse.toarray()
dense = np.full(sparse.shape, np.inf)
dense[sparse.row, sparse.col] = sparse.data
sparse = sparse.tocsr()
row_ind, col_ind = linear_sum_assignment(dense)
print(dense[row_ind, col_ind].sum())

28.0

In [ ]:
row_ind, col_ind = min_weight_full_bipartite_matching(sparse)
print(sparse[row_ind, col_ind].sum())

28.0